In [ ]:
#import the libary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error
import statistics
from matplotlib.ticker import MaxNLocator
from math import sqrt
import math


np.random.seed(5)

#read the data
df=pd.read_csv('thesis_sample_100_shuffled_w_respect_to_id_Base.csv')

In [ ]:
print(df.info())
df.describe()

In [ ]:
#this converts the panda dataframe into ndarray using function to_numpy
x=df[['i_inj','j_inj','i_pro','j_pro','time']].to_numpy() 

#same as above converted to nd array using to_numpy function
y=df[['cum_co2']].to_numpy() 

pd.DataFrame(x).describe(),pd.DataFrame(y).describe()

In [ ]:
#read and prepare the test dataset 
test_last_timesteps=pd.read_csv('thesis_testing_data_last_timesteps.csv')
test_last_timesteps=test_last_timesteps.dropna()
test_last_timesteps_data=test_last_timesteps[['i_inj','j_inj','i_pro','j_pro','time']].to_numpy()

test_last_timesteps_actual=test_last_timesteps[['cum_co2']].to_numpy()

In [ ]:
#scale the data using minmax
scaler=MinMaxScaler()
train_data=scaler.fit_transform(x) # it is important to fit the scaler into training set only
pd.DataFrame(train_data).describe()

In [ ]:
#scale the testing data 
test_last_timesteps_data=scaler.transform(test_last_timesteps_data)
pd.DataFrame(test_last_timesteps_data).describe()

In [ ]:
train_targets=y

pd.DataFrame(train_targets).describe()

In [ ]:
def converter(params):
    #convert the best solution into a layer sizes, integer values from floats
    # transform the layer sizes from float (possibly negative) values into hiddenLayerSizes tuple:
    if round(params[3]) <= 0:
        hiddenLayerSizes = round(params[2]),
    elif round(params[4]) <= 0:
        hiddenLayerSizes = (round(params[2]), round(params[3]))
    elif round(params[5]) <= 0:
        hiddenLayerSizes = (round(params[2]), round(params[3]), round(params[4]))
    elif round(params[6]) <= 0:
        hiddenLayerSizes = (round(params[2]), round(params[3]), round(params[4]), round(params[5]) )
    elif round(params[7]) <= 0:
        hiddenLayerSizes = (round(params[2]), round(params[3]), round(params[4]), round(params[5]),
                           round(params[6]))
    elif round(params[8]) <= 0:
        hiddenLayerSizes = (round(params[2]), round(params[3]), round(params[4]), round(params[5]),
                           round(params[6]),round(params[7]))
    elif round(params[9]) <= 0:
        hiddenLayerSizes = (round(params[2]), round(params[3]), round(params[4]), round(params[5]),
                           round(params[6]),round(params[7]),round(params[8]))
    elif round(params[10]) <= 0:
        hiddenLayerSizes = (round(params[2]), round(params[3]), round(params[4]), round(params[5]),
                           round(params[6]),round(params[7]),round(params[8]),round(params[9]))
    elif round(params[11]) <= 0:
        hiddenLayerSizes = (round(params[2]), round(params[3]), round(params[4]), round(params[5]),
                           round(params[6]),round(params[7]),round(params[8]),round(params[9]),
                           round(params[10]))
    else :
        hiddenLayerSizes = (round(params[2]), round(params[3]), round(params[4]), round(params[5]),
                           round(params[6]),round(params[7]),round(params[8]),round(params[9]),
                           round(params[10]),round(params[11]))

    return hiddenLayerSizes

In [ ]:
def build_model(individuals):
    # transform the layer sizes from float (possibly negative) values into hiddenLayerSizes tuple:
    hiddenLayerSizes=converter(individuals)
    
    model = keras.Sequential()
    model.add(keras.layers.Dense(hiddenLayerSizes[0],input_dim=train_data.shape[1],activation='relu'))
    
    
    if len(hiddenLayerSizes)>1:
        for i in range(len(hiddenLayerSizes)-1):
            model.add(keras.layers.Dense(hiddenLayerSizes[i+1],activation='relu')) #hidden layer generator loop
    else:
        pass
    
    learning_rate=individuals[1]
    
    model.add(keras.layers.Dense(1))
    model.compile(loss='mse', metrics=['mae'],
                  optimizer=tf.keras.optimizers.Adam(learning_rate))  
    
    model.summary()
    
    return model

In [ ]:
#optimal architecture

parameters=[ 32, 1e-03,
             96, 24, 90, 67, 60,-1]

In [ ]:
def k_fold_validation(individual):
 
    num_epochs = 2000
    k = 4
    num_val_samples = len(train_data) // k
    print('num_val_samples:',num_val_samples)
    all_scores = []
    all_mae_histories = []
    
    batch_size=round(individual[0])
    print ('batch_size:',batch_size)
    
    for i in range(k):
        print(f"Processing fold #{i}")    
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]     
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]    
        partial_train_data = np.concatenate([train_data[:i * num_val_samples],         
                                                 train_data[(i + 1) * num_val_samples:]],axis=0)    
        partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],         
                                                    train_targets[(i + 1) * num_val_samples:]],axis=0)   
        
        model = build_model(individual)
    
        history = model.fit(partial_train_data, partial_train_targets,validation_data=(val_data, val_targets),
                            epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=None)  
        
        val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
        all_scores.append(val_mae)
        
        mae_history = history.history["val_mae"]    
        all_mae_histories.append(mae_history)
        
    f=np.mean(all_scores)
    all_mae_histories
    return batch_size, num_epochs, all_mae_histories

In [ ]:
batch_size, num_epochs, all_mae_histories= k_fold_validation(parameters)

In [ ]:
#this code return the average of scores at each epoch among all validation scores
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
#plotting full mae history
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

In [ ]:
#plotting truncated MAE history
truncated_mae_history = average_mae_history[700:15000]
plt.plot(range(1, len(truncated_mae_history) + 1), truncated_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

In [ ]:
#getting the epoch number of lowest mean average MAE
min_mean_average_mae=np.amin(average_mae_history)
best_epoch_number=average_mae_history.index(min_mean_average_mae)+1
print ('best_epoch_number:',best_epoch_number)

In [ ]:
#train the model with best epoch number
model = build_model(parameters)
history= model.fit(train_data, train_targets,epochs=best_epoch_number,
          batch_size=batch_size, verbose=2)
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
# this saves the model that is used to generate the figures for thesis

model.save('Base_model')


In [ ]:
# this loads the model you just saved above

model_saved = tf.keras.models.load_model("Base_model")

In [ ]:
#this function will plot crossplots

def plot(y,ypred):
    
    #the following will get the shape of the input ytest or ypred
    x=([len(l) for l in ypred])[1]
    for i in range(0,x):
        plt.rcParams["figure.figsize"] = (5,5) #change the figure size here
        a = plt.axes(aspect='equal')
        plt.scatter(y[:,i], ypred[:,i],marker='s',color='b')
        plt.xlabel('Cum. CO2 simulation (1e9 m3)')
        plt.ylabel('Cum. CO2 ANN (1e9 m3)')
        ls = [0, np.amax(y[:,i])]
        plt.xlim(ls)
        plt.ylim(ls)
        plt.plot(ls, ls, color='red')
        plt.suptitle('Base Model')
        plt.savefig('figure_cross_plot_base_model.png',dpi=330, bbox_inches='tight')
        plt.show()
        
def metrcs(y,ypred):
    #the following will get the shape of the input ytest or ypred
    x=([len(a) for a in ypred])[1]
    
    for i in range(0,x):
        MAE=metrics.mean_absolute_error(y[:,i],ypred[:,i])
        R2=metrics.r2_score(y[:,i],ypred[:,i])
        MAPE=metrics.mean_absolute_percentage_error(y[:,i],ypred[:,i])
        print ('MAE=',MAE,'R2=',R2, "MAPE=",MAPE)
        
def relative_error(y,ypred):
    
    x=([len(l) for l in ypred])[1]
    for i in range(0,x):
        error=abs(y[:,i]-ypred[:,i])/y[:,i]
        min_error=np.amin(error)
        mean=np.mean(error)
        median=np.median(error)
        max_error=np.amax(error)
        std_dev=statistics.stdev(error)
        print('min=',min_error,'mean=',mean,'median=',median,'max=',max_error, "std_dev=",std_dev)
        plt.scatter(np.arange(1,len(y)+1).reshape(len(y),1), error,marker='o', color='r',s=50)
        plt.xlabel('Test data points')
        plt.ylabel('Absolute relative error')
        lsy=[min_error, max_error]
        lsx=[0,len(y)+1]
        plt.xlim(lsx)
        plt.ylim(lsy)
        plt.grid(False)
        plt.suptitle('Base Model')
        plt.savefig('figure_mean_rel_error_per_experiment_base_Model.png',dpi=330, bbox_inches='tight')
        plt.show()
        plt.figure(5)
        plt.suptitle('Base Model')
        plt.hist(error, bins=5)
        plt.xlabel('Prediction Error')
        plt.xlim([min_error, max_error])
        plt.ylabel('Count')
        plt.grid(False)    
        plt.savefig('figure_histogram_base_model.png',dpi=330, bbox_inches='tight')

def mean_relative_error(y,ypred):
    container=np.array([])
    x=([len(l) for l in ypred])[1]
    for i in range(0,x):
        error=abs(y[:,i]-ypred[:,i])/y[:,i]
        #this outer loop will loop over the timesteps
        for timestep in range(71):  
            #datasets contains 71 timestep each. inner loop will extract the same timesteps for all dataset
            #for ex: there are 20 data for testing dataset which have a timestep of 1
            for dataset in range(0,len(error),71): 
                container=np.append(container,error[timestep+dataset])
        mean_rel_err=np.array([])
        #this loop is for taking the average of the error at each timesteps
        #step size will be the total number of datapoints in each timestep. ex: for 20 dataset step size will be 20
        for i in range(0,len(container),int(len(container)/71.0)): 
            mean_rel_err=np.append(mean_rel_err,np.mean(container[i:i+int(len(container)/71.0),]))
        plt.figure(6)
        plt.scatter(np.arange(1,72,1),mean_rel_err,c='r')
        plt.xlabel('Timestep number')
        plt.ylabel('Mean relative error')
        plt.grid(False)
        plt.show()
        
def mean_rel_error_per_experiment(y,ypred): #this function plots the mean error values with respect to experiment id 
    x=([len(l) for l in ypred])[1]
    for i in range(0,x):
        mean_rel_err=np.array([])
        error=abs(y[:,i]-ypred[:,i])/y[:,i]
        for i in range(0,len(error),71): 
            mean_rel_err=np.append(mean_rel_err,np.mean(error[i:i+71]))
        
        plt.figure(7)
        plt.scatter(np.arange(1,len(error)/71+1,1),mean_rel_err,c='r')
        plt.xlabel('Experiment number')
        plt.ylabel('Mean relative error')
        plt.xticks(np.arange(1,len(error)/71+1,3))
        plt.suptitle('Base Model')
        plt.grid(False)
        plt.show()

In [ ]:
#predictions for training data
y_pred=model.predict(train_data)
#crossplot
plot(y,y_pred)
#metrics for training data
metrcs(y,y_pred)
#error plots for training data
relative_error(y,y_pred)
mean_relative_error(y,y_pred)
mean_rel_error_per_experiment(y,y_pred)

In [ ]:
#plotting the crossplot for testing dataset
y_test_pred = model.predict(test_data)
#crossplot
plot(y_test_actual,y_test_pred)
#metrics
metrcs(y_test_actual,y_test_pred)
#error plots
relative_error(y_test_actual,y_test_pred)
mean_relative_error(y_test_actual,y_test_pred)
mean_rel_error_per_experiment(y_test_actual,y_test_pred)

In [ ]:
#prediction of last timesteps
y_test_last_timesteps_pred = model.predict(test_last_timesteps_data)
#cross plot
plot(test_last_timesteps_actual,y_test_last_timesteps_pred)
#metrics
metrcs(test_last_timesteps_actual,y_test_last_timesteps_pred)
#error plots
relative_error(test_last_timesteps_actual,y_test_last_timesteps_pred)
mean_rel_error_per_experiment(test_last_timesteps_actual,y_test_last_timesteps_pred)

#writing out the prediction and actual values
#convert to dataframe
actual=pd.DataFrame(test_last_timesteps_actual)
predicted=pd.DataFrame(y_test_last_timesteps_pred)
actual.columns=['actual_values']
predicted.columns=['predicted_values']
merged=pd.concat([actual, predicted], axis=1)
merged.to_excel('actual_predicted_base_model.xlsx')

In [ ]:
#this cell is for reading the files. 
permxpor_file=pd.read_csv('perm_x_por_sum.csv')
permxpor_file=permxpor_file[['i','j','permxpor']].to_numpy() 
boundary_grid_file=pd.read_csv('boundary_grid.csv')
boundary_grid_file=boundary_grid_file[['I','J']].to_numpy()
boundary_file=pd.read_csv('boundary_constraint.csv')
boundary_file=boundary_file[['I','J','boundary_with_fault_zones']].to_numpy()

In [ ]:
"""Generator functions start here!!!"""
def boundary_constraint(params,boundary_file): 
    constraint = np.array([])
    for out in range(0,3,2):  #out iterates between inj and pro grid numbers
        for rows in range(len(boundary_file)):
            if round(params[out+1])==boundary_file[rows,1]: #compare j values
            #print (perm[rows])
                if round(params[out])==boundary_file[rows,0]:
                    constraint=np.append(constraint,boundary_file[rows,2])
    
    if sum(constraint)==2.0:
        multiplier=1.0
    else:
        multiplier=0.7  #these are the fitness multiplier to penalize out of boundary individuals
    
    return multiplier

def permxpor_generator(params,permxpor_file):
    permxpor_container = np.array([]) #empty array used as a container of permxpor
    for out in range(0,3,2):
        #print(out)#out iterates between inj and pro grid numbers
        for rows in range(len(permxpor_file)):
            #print (rows) #len function is working properly
            if round(params[out+1])==permxpor_file[rows,1]: #compare j values
                #print (permxpor_file[rows])
                if round(params[out])==permxpor_file[rows,0]: #compare i values
                    permxpor_container=np.append(permxpor_container,
                                                 permxpor_file[rows,2]) #return the matched permxpor value
                    
    return permxpor_container

#min distance generator for inj grid address only, if you want to make for pro grid one change the params in formula 
def min_distance_generator(params, boundary_grid_file): 
    distance_container=np.array([])
    for row in range(len(boundary_grid_file)):
        distance=sqrt((boundary_grid_file[row,0]-round(params[0]))**2+(boundary_grid_file[row,1]-round(params[1]))**2)
        distance_container=np.append(distance_container,distance)
    min_distance=np.amin(distance_container)
    
    return min_distance

def distance_generator(params):
    distance=math.sqrt((round(params[0])-round(params[2]))**2+(round(params[1]-round(params[3])))**2)
    return distance

"""Normalizer function starts here!!!"""

def normalizer(params):
    i_inj=(round(params[0])- 1.00)/(95.0- 1.00)
    j_inj=(round(params[1])- 1.00)/(152.00- 1.00)
    i_pro=(round(params[2])- 1.00)/(95.0- 1.00)
    j_pro=(round(params[3])- 1.00)/(152.00- 1.00)
    time=1.00
    params=np.array([i_inj,j_inj,i_pro,j_pro,time])
    return params

'''update the min max values'''
#           'i_inj',     'j_inj',       'i_pro',       j_pro'     'time'
#                0            1            2            3            4   
# min       1.000000     1.000000     1.000000     1.000000    31.000000      
# max      95.000000   152.000000    95.000000   152.000000  2161.000000   

    
def permxpor_normalizer(perms):
    perms[0]=(perms[0]-  41.106744 )/( 394.800930-  41.106744 ) 
    perms[1]=(perms[1]- 38.693733)/( 427.480096- 38.693733)
    
    return perms,

def min_distance_normalizer(min_distance):
    min_distance=(min_distance-0.000000)/( 35.777088-0.000000)
    return min_distance

def distance_normalizer(distance):
    distance=(distance-5.099020)/(159.062881-5.099020)
    return distance

'''update the min max values'''
#'perm_x_por_inj_sum','perm_x_por_pro_sum','min_distance_inj','distance'
#                  5            6            7            8    
# min      41.106744    38.693733     0.000000     5.099020  
# max     394.800930   427.480096    35.777088   159.062881  

def converter_2(fitness_value):
    cum_co2=fitness_value*10**9
    return cum_co2

In [ ]:
#the results from this optimization run were used in manuscript results
from deap import base
from deap import creator
from deap import tools
import seaborn as sns
from deap import algorithms

def eaSimpleWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm is similar to DEAP eaSimple() algorithm, with the modification that
    halloffame is used to implement an elitism mechanism. The individuals contained in the
    halloffame are directly injected into the next generation and are not subject to the
    genetic operators of selection, crossover and mutation.
    """
    print('The process began')
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)
    

    #iterator is required to iterate in elements of tuple logbook.select("max")
    iterator=0
    #create a numpy array for later use as container and write best individuals to csv file
    all_in_one=np.zeros([1,6])
    all_individuals=np.zeros([1,6])
    
    # Begin the generational process
    for gen in range(1, ngen + 1):
        
        #loop is for printing out the best ind and fitness values of each generation
        for ind in population:
            
            #here i want to print out all the individuals during the optimization
            tuple_ind=tuple(ind)
            tuple_fitness=tuple(ind.fitness.values)
            tuples_to_array=np.asarray((gen,)+tuple_ind+tuple_fitness)
            all_individuals=np.append(all_individuals,tuples_to_array.reshape(1,6),axis=0)
            
            if ind.fitness.values==logbook.select("max")[iterator]: 
                print('best_individual',ind,
                      'Fitness',ind.fitness.values)
                tuple1=tuple(ind) #convert individual into tuple
                tuple2=tuple(ind.fitness.values) #convert individual fitness into tuple
                tuple3=(gen-1,)+tuple1+tuple2 #combine generation, individual, fitnesses into one tuple
                to_array=np.asarray(tuple3)  #convert combined tuple into an array
                reshaped=np.reshape(to_array,(1,6)) #reshape array
                all_in_one=np.append(all_in_one,reshaped,axis=0) #add new values in a new row using axis=0 option
                break #print only one ind per best fitness
        iterator+=1
        
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population) - hof_size)

        # Vary the pool of individuals
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # add the best back to population:
        offspring.extend(halloffame.items)

        # Update the hall of fame with the generated individuals
        halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)
    
    return population, logbook


# boundaries well grid block addresses:
            # [i_inj, j_inj, i_pro, j_pro]
BOUNDS_LOW =  [1,       1,     1,      1]
BOUNDS_HIGH = [95,     152,    95,   152]

NUM_OF_PARAMS = len(BOUNDS_HIGH)

# Genetic Algorithm constants:
POPULATION_SIZE = 200
P_CROSSOVER = 0.9  # probability for crossover
P_MUTATION = 0.3   # probability for mutating an individual
MAX_GENERATIONS = 500
HALL_OF_FAME_SIZE = 3
CROWDING_FACTOR = 10.0  # crowding factor for crossover and mutation

# set the random seed:
RANDOM_SEED = 44
random.seed(RANDOM_SEED)

toolbox = base.Toolbox()

# define a single objective, maximizing fitness strategy:
creator.create("FitnessMax_base_model", base.Fitness, weights=(1.0,))

# create the Individual class based on list:
creator.create("Individual_base_model", list,
               fitness=creator.FitnessMax_base_model)

# define the well gridblock addresses individually:
for i in range(NUM_OF_PARAMS):
    # "block_address_0", "block_address_1", ...
    toolbox.register("block_address_" + str(i),
                     random.uniform,
                     BOUNDS_LOW[i],
                     BOUNDS_HIGH[i])

# create a tuple containing an block address generator for each well i-j grid:
block_addresses = ()
for i in range(NUM_OF_PARAMS):
    block_addresses = block_addresses + \
                            (toolbox.__getattribute__("block_address_" + str(i)),)

# create the individual operator to fill up an Individual instance:
toolbox.register("individualCreator",
                 tools.initCycle,
                 creator.Individual_base_model,
                 block_addresses,
                 n=1)

# create the population operator to generate a list of individuals:
toolbox.register("populationCreator",
                 tools.initRepeat,
                 list,
                 toolbox.individualCreator)


# fitness calculation
def accuracy(individual):
    multiplier=boundary_constraint(individual,boundary_file)

    #normalize all values
    individual=normalizer(individual)    

    #reshape individual for ANN model
    individual=np.reshape(individual,(1,5))
    
    #evaluate the fitness function
    accuracy= model(individual)
    fitness=np.array(accuracy[:,0])*multiplier
    
    
    return fitness,


toolbox.register("evaluate", accuracy)

# genetic operators:
toolbox.register("select", tools.selTournament, tournsize=2)

toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR)

toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low=BOUNDS_LOW,
                 up=BOUNDS_HIGH,
                 eta=CROWDING_FACTOR,
                 indpb=1.0/NUM_OF_PARAMS)


# Genetic Algorithm flow:
def main():

    # create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)

    # prepare the statistics object:
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", np.max)
    stats.register("avg", np.mean)

    # define the hall-of-fame object:
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

    # perform the Genetic Algorithm flow with hof feature added:
    population, logbook = eaSimpleWithElitism(population,
                                                      toolbox,
                                                      cxpb=P_CROSSOVER,
                                                      mutpb=P_MUTATION,
                                                      ngen=MAX_GENERATIONS,
                                                      stats=stats,
                                                      halloffame=hof,
                                                      verbose=True)
    #print all of the best solutions found:
    
    #for i in range(0,15):
     #   print (converter(hof.items[i]))
        
    
    # print info for best solution found:
       
    print("-- Best Individual = ", hof.items[0])
    print("-- Best Fitness--= ", hof.items[0].fitness.values[0])
    
    #print converted solutions here below
    
    best = hof.items[0]
    
    cum_co2 = converter_2(hof.items[0].fitness.values[0])
    print("-- Best Individual = ", best)
    print("-- Best Fitness-- Cumulative CO2 = ", cum_co2)

    # extract statistics:
    maxFitnessValues, meanFitnessValues = logbook.select("max", "avg")

    # plot statistics:
    sns.set_style("whitegrid")
    plt.plot(maxFitnessValues, color='red')
    plt.plot(meanFitnessValues, color='green')
    plt.xlabel('Generation')
    plt.ylabel('Max / Average Fitness')
    plt.title('Max and Average fitness over Generations')

    plt.show()

#execute the main function
main()

In [ ]:
# 2nd run with different random seed
RANDOM_SEED = 2
random.seed(RANDOM_SEED)
main()

In [ ]:
# 3rd run with different random seed
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
main()

In [ ]:
# 4th run with different random seed
RANDOM_SEED = 4
random.seed(RANDOM_SEED)
main()

In [ ]:
# 5th run with different random seed
RANDOM_SEED = 5
random.seed(RANDOM_SEED)
main()

In [ ]:
# 6th run with different random seed
RANDOM_SEED = 6
random.seed(RANDOM_SEED)
main()

In [ ]:
# 7th run with different random seed
RANDOM_SEED = 7
random.seed(RANDOM_SEED)
main()

In [ ]:
# 8th run with different random seed
RANDOM_SEED = 8
random.seed(RANDOM_SEED)
main()

In [ ]:
# 9th run with different random seed
RANDOM_SEED = 9
random.seed(RANDOM_SEED)
main()

In [ ]:
# 10th run with different random seed
RANDOM_SEED = 10
random.seed(RANDOM_SEED)
main()